# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [7]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)
sum(data[data.race=='w'].call)

235.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

In [8]:
w = data[data.race=='w']
b = data[data.race=='b']

# Question 1 - What test is appropriate for this problem? Does CLT apply?¶

We need to consider a few key factors to answer this question successfully:

1. What are we trying to answer? In this case, we are looking at doing a comparison between two distinct populations.
2. Is our sample size larger than 30? Yes. 

With this information, we can say that it we should be pursuing a Z-test. The Central Limit Theorem states that repeated samples from the population, will eventually lead to a true population mean. In this context, the CLT can be applied if we aren't confident that we have a normal population.

# Q2. What are the null and alternate hypotheses?

Null Hypotheses: There is no racial discrimination between black and white candidates

Alternative Hypotheses: There exists racial discrimination between black and white candidates

# Q3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

Let us take a look at what the success rate looks like for black and white candidates. 

In [26]:
w_call_prop = sum(w.call)/(len(w.call))
b_call_prop = sum(b.call)/(len(b.call))

print ("The proportion of white sounding name candidates being called back is", w_call_prop*100, " percent")
print ("The proportion of white sounding name candidates being called back is", b_call_prop*100, "percent")

The proportion of white sounding name candidates being called back is 9.65092402464  percent
The proportion of white sounding name candidates being called back is 6.4476386037 percent


In [31]:
standard_err = np.sqrt(((b_call_prop)*(1-b_call_prop))/len(b) + (w_call_prop*(1 - w_call_prop)/len(w)))
Z_score = 1.96 #Assuming 95% CI
MoE = Z_score * standard_err
print ("The Margin of Error (MoE) is +-", MoE)

The Margin of Error (MoE) is +- 0.0152554063499


In [32]:
variance_w_b = w_call_prop - b_call_prop
variance_w_b

0.032032854209445585

With this, we can clearly see there is a difference between white / black candidates being called. 

However, let us get an exact figure via a 95% CI.

In [41]:
confidence_interval = [ variance_w_b + MoE, variance_w_b - MoE]
print("The CI at a 95 percent band is %0.6F +/- %0.5F " % (variance_w_b, MoE))
print("CVs with white-sounding names that recieve a call is between %0.6F and %0.6F higher than the proportion of CVS with black-sounding names" % (confidence_interval[1],confidence_interval[0]))

The CI at a 95 percent band is 0.032033 +/- 0.01526 
CVs with white-sounding names that recieve a call is between 0.016777 and 0.047288 higher than the proportion of CVS with black-sounding names


In [46]:
# The P value is this context can be calculated as:
from statsmodels.stats.weightstats import ztest
ztest = ztest(data.call[data.race == 'w'],data.call[data.race == 'b'], alternative = 'two-sided')
print("Z-score is equal to : %6.2F  p-value equal to: %6.10F" % (abs(ztest[0]),ztest[1]))

Z-score is equal to :   4.11  p-value equal to: 0.0000387674


Hence with a P value of 0.000038 which is < 0.05, we can see there exists sufficient evidence to reject the Null Hypothesis.

# Q4. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

No, in this context we have only used statistical groundings to compare two distinct possibilities that 'race' is a significant factor with response to call back success.
There are still plenty of other variables in the data set (60+) other variables that we can compare via a Classification And Regression Tree and
from this, identify what other key variables are related to impacting call back success.

Race is just one of many other variables that could be tested.
Further analysis would have me yield a CART analysis, without accounting for multicolinearity (as I am not predicting), to identify what other variables
have a impact on the odds of call back success.
